# 4.4 Other Illustrations of Public Key Cryptography

## Exercises 4.4

In [4]:
%%capture
%run ./4_3_two_examples.ipynb
from sympy import isprime

from __main__ import EncodingScheme, pk_encrypt, pk_decrypt


### 1. `[26, 12, 22, 58, 61]`. That's the signature of a former Special Agent for the FBI in the 1960s. It has been encoded using the agent's private key, which is none of your business. Verify the signature by using his public key $7$ with $n = 77$. The text-numeric correspondence is the one relating a letter to its alphabetic position. Each integer corresponds to a single letter.

In [2]:
Tuple1Char = Tuple[CharStr, Literal[1]]
class OneCharEncoder(EncodingScheme[Literal[1]]):
    """Single character encoder using alphabetic position one charater at a time."""
    _CHARS = "\0abcdefghijklmnopqrstuvwxyz"
    def pos(self, char: CharStr) -> int:
        if char.lower() not in self._CHARS:
            raise ValueError(f"Expected valid char, got {char}")
        return self._CHARS.index(char.lower())
    def char_at(self, pos: int) -> CharStr:
        index = pos % 26
        if index < 1:
            index += 26
        return self._CHARS[pos]
    def encode(self, chars: Tuple1Char) -> int:
        if len(chars) != 1 or len(chars[0]) != 1:
            raise ValueError(f"Expected valid 1 char tuple, got {chars}")
        return self.pos(chars[0])
    def decode(self, encoded_chars: int) -> Tuple1Char:
        return (self.char_at(encoded_chars))
    def block_size(self) -> int:
        return 1
encoder_1 = OneCharEncoder()

In [3]:
sig_1 = [26, 12, 22, 58, 61]
verified_sig = pk_decrypt(7, 77, encoder_1, sig_1)
print(f"Verified signature: {verified_sig}")

Verified signature: elvis


In [11]:
class DiffieHellmanExchange:
    """ Diffie-Hellman Key Exchange"""
    def __init__(self, p:int, q:int):
        if p < 2 or not isprime(p):
            raise ValueError(f"Expected prime p, p>2, got {p}")
        self.p = p
        if not 1 < q < p:
            raise ValueError(f"Expected q, 1<q<p, got {q}")
        self.q = q
    def init_msg(self, key:int) -> int:
        """ Compute the initial message for a DF exchange """
        if not 2 < key < self.p:
            raise ValueError(f"Expected key, 2<key<p, got {key}")
        return pow(self.q, key, self.p)
    def final_msg(self, msg:int, key:int) -> int:
        """ Compute the intermediate message for a DF exchange """
        if not 2 < key < self.p:
            raise ValueError(f"Expected key, 2<key<p, got {key}")
        if not 0 < msg < self.p:
            raise ValueError(f"Expected msg, 0<msg<p, got {msg}")
        return pow(msg, key, self.p)


### 2. Two individuals decide to use the Diffie-Hellman Key Exchange System to communicate a keyword. They agree that $p = 11$ and $q = 8$. One chooses private key $3$ and the other private key $4$. Verify that they each wind up with the same keyword.

In [17]:
df = DiffieHellmanExchange(p=11, q=8)
A_key = 3
B_key = 4

A_init_msg = df.init_msg(A_key)
print(f"A's init msg: {A_init_msg}")

B_init_msg = df.init_msg(B_key)
print(f"B's init msg: {B_init_msg}\n")

print(f"A's final msg: {df.final_msg(B_init_msg, A_key)}")
print(f"B's final msg: {df.final_msg(A_init_msg, B_key)}")


A's init msg: 6
B's init msg: 4

A's final msg: 9
B's final msg: 9


### 3. Repeat Exercise #2 except this time assume that $p = 4653848293$, $q = 65478390$, one's private key is $76350293$ and the other's is $233451876$.

In [18]:
df = DiffieHellmanExchange(p=4653848293, q=65478390)
A_key = 76350293
B_key = 233451876

A_init_msg = df.init_msg(A_key)
print(f"A's init msg: {A_init_msg}")

B_init_msg = df.init_msg(B_key)
print(f"B's init msg: {B_init_msg}\n")

print(f"A's final msg: {df.final_msg(B_init_msg, A_key)}")
print(f"B's final msg: {df.final_msg(A_init_msg, B_key)}")

A's init msg: 2721419856
B's init msg: 4077302202

A's final msg: 2014163551
B's final msg: 2014163551



### 4. One individual wants to send a message consisting of the single number $25$ to another using the Massey-Omura System. Both agree on the prime $p = 31$. The sender chooses two private keys: $13$ and $7$; the receiver chooses two private keys: $17$ and $23$. Verify that the receiver actually receives the number $25$.

In [31]:
def msg_to_nums(msg: str) -> list[int]:
    """ Convert a message to a list of numbers """
    return [encoder_1.encode(c) for c in msg]

In [32]:
p = 31

sender_e = 13
sender_d = 7

receiver_e = 17
receiver_d = 23

msg = [25]

msg1 = msg_to_nums(pk_decrypt(sender_e, p, encoder_1, msg))
print(f"Sender's first encrypted message: {msg1}")

msg2 = msg_to_nums(pk_decrypt(receiver_e, p, encoder_1, msg1))
print(f"Receiver's intermediate encrypted message: {msg2}")

msg3 = msg_to_nums(pk_decrypt(sender_d, p, encoder_1, msg2))
print(f"Sender's second encrypted message: {msg3}")

msg4 = msg_to_nums(pk_decrypt(receiver_d, p, encoder_1, msg3))
print(f"Receiver's decrypted message: {msg4}")

Sender's first encrypted message: [25]
Receiver's intermediate encrypted message: [5]
Sender's second encrypted message: [5]
Receiver's decrypted message: [25]


### 5. Repeat Exercise #4 except this time assume that $p = 4658349003443$, the sender's private keys are $65477368573$ and $3513574598399$ and the receiver's private keys are $763485413$ and $4441762259265$. And the message is: `Joplin`.

In [44]:
# use a 7 bit ascii encoder for this exercise.  Will preserve capitalization
# and get six chars in a block for p like 4658349003443 (which requires 43 bits)
Ascii7BitEncoderTupleChars = Tuple[CharStr, Literal[6]]
class Ascii7BitEncoder(EncodingScheme[Literal[6]]):
    def __init__(self):
        self._BLOCKSIZE = 6
    def encode(self, chars: Ascii7BitEncoderTupleChars) -> int:
        if len(chars) != self._BLOCKSIZE:
            raise ValueError(f"chars must be of length {self._BLOCKSIZE}, not {len(chars)}")
        # Take each of the chars, get its ascii code and build the block
        # block is reversed so we can shift out more naturally during decode
        result = 0
        for i in range(len(chars) - 1, -1, -1):  # reverse order
            if ord(chars[i]) > 127:
                # raise an error if input contains character that is not 7 bits
                raise ValueError(f"character {chars[i]} is not in range(256)")
            result = (result << 7) | (ord(chars[i]) & 0x7F)
        return result
    def decode(self, encoded_chars: int) -> AsciiEncoderTupleChars:
        chars = []
        for _ in range(self._BLOCKSIZE):
            chars.append(chr(encoded_chars & 0x7F))
            encoded_chars >>= 7
        return tuple(chars)
    def block_size(self) -> int:
        return self._BLOCKSIZE
encoder_5 = Ascii7BitEncoder()

In [51]:
p = 4658349003443

sender_e = 65477368573
sender_d = 3513574598399

receiver_e = 763485413
receiver_d = 4441762259265

msg = "Joplin"

msg1 = pk_encrypt(sender_e, p, encoder_5, msg)
print(f"Sender's first encrypted message: {msg1}")

msg2 = pk_decrypt(receiver_e, p, encoder_5, msg1)
print("Receiver's intermediate encrypted message: " +
      f"{[encoder_5.encode(tuple(msg2))]}")

msg3 = pk_encrypt(sender_d, p, encoder_5, msg2)
print(f"Sender's second encrypted message: {msg3}")

msg4 = pk_decrypt(receiver_d, p, encoder_5, msg3)
print(f"Receiver's decrypted message: '{msg4}'")

Sender's first encrypted message: [2041298202570]
Receiver's intermediate encrypted message: [4151522335983]
Sender's second encrypted message: [1825433649931]
Receiver's decrypted message: 'Joplin'



### 6. In practice, messages are often encrypted several times before being sent. Suppose Beth sends a message to Stephanie that Beth first encrypts twice: once using Stephanie's _public_ key (as in the RSA algorithm) and once using Beth's _private_ key. In other words, assuming

 | Entity                      | Symbol       |
 |-----------------------------|--------------|
 | Beth's message to Stephanie | $m$          |
 | Beth's private key          | $d_B$        |
 | Beth's public key           | $e_B$, $n_B$ |
 | Stephanie's private key     | $d_S$        |
 | Stephanie's public key      | $e_S$, $n_S$ |

 Beth's enciphered message is $m' = (m^{e_S} \pmod{n_S})^{d_B} \pmod{n_B}$. Explain
 how Stephanie would go about deciphering $m'$.

First, Stephanie encrypts $m'$ using Beth's public key, $e_B$ and $n_B$, then she decrypts the result of that with her own decrypt key, $d_S$ and $n_S$.  The result is the original message, as shown here.

$$
\begin{aligned}
       \bigg[ \Big[(m^{e_S} \pmod{n_S})^{d_B} \pmod{n_B}\Big]^{e_B} \pmod{n_B} \bigg]^{d_S} \pmod{n_S} &=
          \Big[ (m^{e_S} \pmod{n_S})^{d_Be_B} \pmod{n_B} \Big]^{d_S} \pmod{n_S} \\
         &= \left[ m^{e_S} \pmod{n_S} \right]^{d_S} \pmod{n_S} \\
         &= m^{e_Sd_S} \pmod{n_S} \\
         &= m
\end{aligned}
$$